In [1]:
import redis
import time
from openai import OpenAI
import torch
from sentence_transformers import util, SentenceTransformer
import json
import requests
from IPython.display import Markdown, display


/home/ahmed/Desktop/AGH_Data_Agency_Holding_SA/Projet_LLM_Server/Server_LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#========================================
# configurations
#========================================

r = redis.Redis(host='localhost', port=6379, db=0)
API_URL = "http://localhost:11434/api/generate"
EMBEDDING_MODEL_NAME = "multi-qa-mpnet-base-dot-v1"
DATA = torch.load("/home/ahmed/Desktop/AGH_Data_Agency_Holding_SA/Projet_LLM_Server/data/meal_embeddings.pt")
EMBEDDINGS_MEAL = DATA["embeddings"]
TEXTS_MEAL = DATA["texts"]
SIMILARITY_THRESHOLD = 0.85


In [3]:
def retrieve_relevant_resources_person(
    query: str,
    embeddings: torch.tensor,
    model_name: str = EMBEDDING_MODEL_NAME,
    n_resources_to_return: int = 5,
):
    model = SentenceTransformer(model_name, device="cpu")
    query_embedding = model.encode(query, convert_to_tensor=True)

    dot_scores = util.dot_score(query_embedding, embeddings)[0]

    scores, indices = torch.topk(dot_scores, k=n_resources_to_return)

    return indices.tolist()

In [4]:
def prompt_formatter_person(query: str, context_items: list[str]) -> str:
    context = "- " + "\n- ".join(context_items)
    
    base_prompt = f"""
    You are an expert cooking assistant.

    Below are recipes from the database:
    {context}

    User question:
    {query}

    Answer using the database information when possible.
    If there is no relevant recipe, give a general cooking answer.
    """

    # print(f"the context is :{base_prompt}")
    return base_prompt

In [5]:
def llm_response(ollama_model_name, prompt):
    data = {
        "model": ollama_model_name,
        "prompt": prompt
    }
    
    response_text = ""
    response = requests.post(API_URL, json=data, stream=True)
    
    for line in response.iter_lines():
        if line:
            text = json.loads(line)["response"]
            response_text += text  
    
    return response_text  


In [6]:
def automate_asking(query, ollama_model_name):
    indices = retrieve_relevant_resources_person(query=query, embeddings=EMBEDDINGS_MEAL)
    context_items = [TEXTS_MEAL[i] for i in indices]

    prompt = prompt_formatter_person(query=query, context_items=context_items)
    return llm_response(ollama_model_name, prompt)


In [7]:
def get_embedding(text: str):
    sim_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device="cpu")
    return sim_model.encode(text, convert_to_tensor=True)

In [8]:
def find_similar_question(query_emb):
    for key in r.keys():
        raw = r.get(key)
        if not raw:
            continue  

        try:
            cached_data = json.loads(raw)
        except json.JSONDecodeError:
            continue  

        cached_emb = torch.tensor(cached_data["embedding"])
        similarity = util.cos_sim(query_emb, cached_emb).item()
        if similarity >= SIMILARITY_THRESHOLD:
            return key.decode(), cached_data["response"]
    return None, None


In [9]:
def ask_llm_with_redis_smart(question):
    query_emb = get_embedding(question)
    
    similar_key, cached_response = find_similar_question(query_emb)
    
    if cached_response:
        print("Using cached response for a similar question!")
        from_cache = True
        response_text = cached_response
    else:
        response_text = automate_asking(question, ollama_model_name="llama3.2")
        data_to_cache = {
            "embedding": query_emb.tolist(), 
            "response": response_text
        }
        r.set(question, json.dumps(data_to_cache))
        from_cache = False
    
    return response_text, from_cache

In [ ]:
start = time.time()
res, from_cache = ask_llm_with_redis_smart("can you tell me a Dessert recette Banana Pancakes")
print("The LLM Resposne is :\n")
display(Markdown(res))
print("Cache:", from_cache)
print("Temps:", time.time() - start)


print("\n\n\n############################################################\n\n\n")

start = time.time()
res, from_cache = ask_llm_with_redis_smart("i want a Dessert recette Banana Pancakes")
print("The LLM Resposne is :\n")
display(Markdown(res))
print("Cache:", from_cache)
print("Temps:", time.time() - start)


The LLM Resposne is :



A delicious dessert! According to our database, I have just the recipe for you: Banana Pancakes!

Here's how to make it:

Ingredients:
- 1 large banana
- 2 medium eggs
- A pinch of baking powder
- A sprinkling of vanilla extract
- 1 tsp oil
- 25g pecan nuts
- 125g raspberries

Instructions:
1. Mash the banana with a fork until it resembles a thick purée.
2. Stir in the eggs, baking powder, and vanilla extract.
3. Heat a large non-stick frying pan or pancake pan over medium heat and brush with half the oil.
4. Using half the batter, spoon two pancakes into the pan, cook for 1-2 mins each side, then tip onto a plate.
5. Repeat the process with the remaining oil and batter.
6. Top the pancakes with pecans and raspberries.

Enjoy your Banana Pancakes!

(Note: I couldn't find any additional information about this recipe in our database. If you'd like more guidance or variations, feel free to ask!)

Cache: False
Temps: 204.81155562400818



############################################################



Using cached response for a similar question!
The LLM Resposne is :



A delicious dessert! According to our database, I have just the recipe for you: Banana Pancakes!

Here's how to make it:

Ingredients:
- 1 large banana
- 2 medium eggs
- A pinch of baking powder
- A sprinkling of vanilla extract
- 1 tsp oil
- 25g pecan nuts
- 125g raspberries

Instructions:
1. Mash the banana with a fork until it resembles a thick purée.
2. Stir in the eggs, baking powder, and vanilla extract.
3. Heat a large non-stick frying pan or pancake pan over medium heat and brush with half the oil.
4. Using half the batter, spoon two pancakes into the pan, cook for 1-2 mins each side, then tip onto a plate.
5. Repeat the process with the remaining oil and batter.
6. Top the pancakes with pecans and raspberries.

Enjoy your Banana Pancakes!

(Note: I couldn't find any additional information about this recipe in our database. If you'd like more guidance or variations, feel free to ask!)

Cache: True
Temps: 5.0426716804504395
